<a href="https://colab.research.google.com/github/MSaber9/DL/blob/master/VehiclesCls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratory work №3. Deep neural networks for image classification

> The task is to build the convolutional neural network for vehicles images recognizion. There is three classes of objects to recognize: personal transport, ambulance, police cars.



---





## Library

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , MaxPool2D , UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
from time import time

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.layers import Input, Concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


## Preprocessing Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_path_X = 'drive/My Drive/Colab Notebooks/Data/vehicleimg'
root_path_y = 'drive/My Drive/Colab Notebooks/Data/vehicletrgt'


In [0]:
# for reading also binary mode is important 
import pickle

dbfile = open(root_path_X, 'rb')      
X = pickle.load(dbfile) 

dbfile = open(root_path_y, 'rb')      
y = pickle.load(dbfile) 
y-= min(y)

In [0]:
np.unique(y)

array([0, 1, 2])

In [0]:
X.shape , y.shape

((13785, 186, 250, 3), (13785,))

In [0]:
# --------------------- Dowmscale to reduce size ---------------------------

# import matplotlib.pyplot as plt
# from skimage import data, color
# from skimage.transform import rescale, resize, downscale_local_mean

# X_gray = color.rgb2gray(X)
# X_rescale = rescale(X_gray, 0.25, anti_aliasing=False)

In [0]:
np.where(y==2)

(array([ 9461,  9462,  9463, ..., 13782, 13783, 13784]),)

In [0]:
# Split data to x, y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04, random_state=42)

X_train.shape , X_test.shape , y_train.shape , y_test.shape

((13233, 186, 250, 3), (552, 186, 250, 3), (13233,), (552,))

In [0]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train1.shape , X_test1.shape , y_train1.shape , y_test1.shape

((10586, 186, 250, 3), (2647, 186, 250, 3), (10586,), (2647,))

In [0]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train1, y_train1, test_size=0.2, random_state=42)

X_train2.shape , X_test2.shape , y_train2.shape , y_test2.shape

((8468, 186, 250, 3), (2118, 186, 250, 3), (8468,), (2118,))

In [0]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train2, y_train2, test_size=0.2, random_state=42)

X_train3.shape , X_test3.shape , y_train3.shape , y_test3.shape

((6774, 186, 250, 3), (1694, 186, 250, 3), (6774,), (1694,))

In [0]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_train3, y_train3, test_size=0.2, random_state=42)

X_train4.shape , X_test4.shape , y_train4.shape , y_test4.shape

((5419, 186, 250, 3), (1355, 186, 250, 3), (5419,), (1355,))

In [0]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X_train4, y_train4, test_size=0.2, random_state=42)

X_train5.shape , X_test5.shape , y_train5.shape , y_test5.shape

((4335, 186, 250, 3), (1084, 186, 250, 3), (4335,), (1084,))

In [0]:
np.unique(y_train5)

array([0, 1, 2])

In [0]:
np.unique(y_test5)

array([0, 1, 2])

In [0]:
from keras_preprocessing.image import ImageDataGenerator

In [0]:
# ------------------- One Hot Encoding --------------------------

# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train5 = np_utils.to_categorical(y_train5, num_classes=3)
y_test5 = np_utils.to_categorical(y_test5, num_classes=3)

# ------------------- ImageDataGenerator --------------------------

datagen = ImageDataGenerator(zoom_range = 0.25, brightness_range = (0.3, 1.3), rescale = 1/255)
vdatagen = ImageDataGenerator(rescale = 1/255)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train5)
vdatagen.fit(X_test5)

In [0]:
#y_train1 = y_train1
#y_test1 = y_test1

In [0]:
# ------------------- Normalization --------------------------
#from skimage import exposure # data
#X_train1 = exposure.equalize_hist(X_train1)
#X_test1 = exposure.equalize_hist(X_test1)

## Create Model CNN

In [0]:
# Define Parameter
batch_size = 32
num_classes = 3
epochs = 100

img_rows, img_cols = 186, 250
input_shape = X_train1[0].shape

In [0]:
def CreateCNN(NeuroFeatures = 64 , NeuroInput = 30 , ActivFun = 'relu' , init ='he_normal' , optimizer = 'adam', DropoutRate = 0.055):   
    
      model = Sequential()

      model.add(Conv2D(32, kernel_size=(3, 3), activation=ActivFun, kernel_initializer=init, input_shape=input_shape))
      model.add(MaxPooling2D((2, 2)))
      #model.add(Dropout(0.25))

          # ------------------------------------------------ Features Learning ----------------------------------------------------------------------
        
      #NeuroFeatures = [64,128]
      #DropoutRate = [0.25,0.4]
      #ActivFun = 'relu'
      layerNo = 2

      for i in range (layerNo):
            model.add(Conv2D(NeuroFeatures, (3, 3), activation=ActivFun))
            model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
            model.add(Dropout(DropoutRate))

            
          # ------------------------------------------------- Classification ------------------------------------------------------------------------

      model.add(Flatten())
      model.add(Dropout(0.3))
      model.add(Dense(128, activation=ActivFun))
      model.add(Dropout(0.05))

          # ---------------------------------------------------- Output -----------------------------------------------------------------------------

      model.add(Dense(num_classes, activation='softmax'))

          # ------------------------------------------------ Compile Model --------------------------------------------------------------------------

      model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
      # sparse_categorical_crossentropy / categorical_crossentropy

      model.summary()

          # ---------------------------------------------- Train the model ---------------------------------------------------------------------------

      history = model.fit_generator(datagen.flow(X_train5, y_train5, batch_size=64),
                                    steps_per_epoch=len(X_train5) / 64, epochs=epochs, validation_data = vdatagen.flow(X_test5, y_test5, batch_size=64))

      score = model.evaluate(X_test5, y_test5, verbose=0)
      
      return (model)


In [0]:
History1 = CreateCNN(NeuroFeatures = 64 , NeuroInput = 32 , ActivFun = 'relu' , init ='he_normal' , optimizer = 'Adam', DropoutRate = 0.025)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 184, 248, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 92, 124, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 90, 122, 64)       18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 44, 60, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 44, 60, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 42, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 20, 28, 64)       

In [0]:
print(History1.evaluate(X_test1/255, y_test1, verbose=0))

[0.5242172233984667, 0.9009009009009009]


## GrideSerchCV CNN

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# ----------------------------- Wrappers ---------------------------------------

start=time()
model = KerasClassifier(build_fn=CreateCNN)

# ----------------------------- Parameters -------------------------------------
#CreateCNN(NeuroFeatures = 64 , NeuroInput = 32 , ActivFun = 'relu' , init ='he_normal' , optimizer = 'adam', DropoutRate = 0.25)

#NeuroFeatures = [64,128]
#NeuroInput = [32,64]
ActivFun = ['relu', 'sigmoid']
init = ['he_normal', 'he_uniform']
optimizer =  ['Adadelta','Adam']
#DropoutRate = [0.0 ,0.1,0.2]

param_grid = dict( ActivFun = ActivFun , init = init , optimizer = optimizer)

# ------------------------------ GridSearchCV ----------------------------------

grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# -------------------------------- Results -------------------------------------

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 184, 248, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 92, 124, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 90, 122, 64)       18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 44, 60, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 44, 60, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 42, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 20, 28, 64)       

KeyboardInterrupt: ignored